In [142]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# class ESGModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(ESGModel, self).__init__()
#         self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         _, hidden = self.gru(x)
#         output = self.fc(hidden[-1])
#         return output
# input_size = ...  # Размер входного вектора текста после векторизации
# hidden_size = ...  # Размер скрытого состояния GRU
# output_size = 1 

# model = ESGModel(input_size, hidden_size, output_size)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# X_train, X_test, y_train, y_test = train_test_split(text_data, esg_ratings, test_size=0.2, random_state=42)

# # Обучение модели
# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train)
#     loss = criterion(outputs, y_train)
#     loss.backward()
#     optimizer.step()

# # Оценка модели
# model.eval()
# with torch.no_grad():
#     test_outputs = model(X_test)
#     test_loss = criterion(test_outputs, y_test)
#     print(f"Test Loss: {test_loss.item()}")

In [133]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.9 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 7.9 MB/s eta 0:00:00m eta 0:00:010:01:01
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [90]:
import pandas as pd
data = pd.read_excel('ESG_Ranking_2022.xlsx')

In [91]:
data = data.dropna()

In [121]:
data['ESG Rating band'].unique()

array([8, 7, 6, 5, 4, 3, 2, 1])

In [112]:
score_mapping = {'AAA ': 9, 'AA ': 8, 'A ': 7, 'BBB ': 6, 'BB ': 5, 'B ': 4, 'CCC ': 3, 'CC ': 2, 'C ': 1, 'M ': 1}
data = data.replace(score_mapping)
data = data.replace({'smissioAn) ': 7, 'ea BBB ': 6, 's BBB ':6,'smissiBoBn) ':5,'smissioBn) ':4,'es CCC ':3,'s CCC ':3,'smissiMon) ':1})

In [113]:
data

,Company,Industry,Sub-Industry,ESG Rank,ESG Rating band,E Rank,E Rating Band,S Rank,S Rating Band,G Rank,G Rating Band,Year of latest asessed report,Type of the ESG rating
0,Sber,Financials,Banks,1,8,2,8,1,9,4,9,2022.0,solicited
2,Phosagro,Chemicals,Agricultural Chemicals,2,8,5,8,3,8,2,9,2022.0,solicited
4,Polyus,Metals&Mining,Precious metals,3,8,1,8,2,8,14,8,2022.0,solicited
6,Tatneft,Oil&Gas,Integrated Oil & Gas,4,8,13,7,5,8,6,8,2022.0,unsolicited
8,Rosneft,Oil&Gas,Integrated Oil & Gas,5,8,4,8,22,7,5,9,2022.0,solicited
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,NOVIKOMBANK,Financials,Banks,156,1,114,1,145,1,155-156,1,2022.0,unsolicited
312,Quadra – Power Generation,Energy,Electric Utilities (Gener,atio1n5a7n-1d6T0ran,1,157-160,1,157-160,1,157-160,1,2022.0,unsolicited
314,Russian Copper Company,Metals&Mining,Copper,157-160,1,157-160,1,157-160,1,157-160,1,2022.0,unsolicited
316,UMMC,Metals&Mining,Specialty Mining & Meta,ls 157-160,1,157-160,1,157-160,1,157-160,1,2022.0,unsolicited


In [146]:
X = data[['E Rating Band','S Rating Band','G Rating Band']]
y = data['ESG Rating band']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'CatBoost' : CatBoostClassifier(random_state=57, verbose=0),
    'Gradient Boosting': GradientBoostingClassifier(random_state=3),
}

param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear']
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 150, 200, 250],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
        'max_depth': [3, 4, 5, 6, 7, 8]
    },
    'CatBoost': {
        'iterations': [50, 100, 150, 200, 250],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
        'depth': [3, 4, 5, 6, 7, 8]
    }
}


best_models = {}
for model_name, model in models.items():
    print(f"Tuning hyperparameters for {model_name}...")
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    y_pred = grid_search.predict(X_test)
    print(f"Accuracy for {model_name}: {accuracy_score(y_test, y_pred)}")

Tuning hyperparameters for Logistic Regression...
Best parameters for Logistic Regression: {'C': 100, 'solver': 'lbfgs'}
Accuracy for Logistic Regression: 0.75
Tuning hyperparameters for CatBoost...
Best parameters for CatBoost: {'depth': 4, 'iterations': 200, 'learning_rate': 0.1}
Accuracy for CatBoost: 0.65625
Tuning hyperparameters for Gradient Boosting...
Best parameters for Gradient Boosting: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50}
Accuracy for Gradient Boosting: 0.625


KeyError: 'Random Forest'